<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Identify Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can4) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [81]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import scipy.sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import xgboost
import lightgbm
import timeit
from datetime import datetime
from tqdm import tqdm

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Считаем данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [3]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = '../data/' 

In [4]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [6]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182793 entries, 1 to 182793
Data columns (total 21 columns):
site1      182793 non-null int64
time1      182793 non-null object
site2      181175 non-null float64
time2      181175 non-null object
site3      179441 non-null float64
time3      179441 non-null object
site4      178054 non-null float64
time4      178054 non-null object
site5      176653 non-null float64
time5      176653 non-null object
site6      175268 non-null float64
time6      175268 non-null object
site7      173960 non-null float64
time7      173960 non-null object
site8      172738 non-null float64
time8      172738 non-null object
site9      171437 non-null float64
time9      171437 non-null object
site10     170247 non-null float64
time10     170247 non-null object
user_id    182793 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 30.7+ MB


In [8]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,9,2014-10-04 12:24:43,304.0,2014-10-04 12:25:34,308.0,2014-10-04 12:28:33,307.0,2014-10-04 12:28:33,91.0,2014-10-04 12:28:33,308.0,2014-10-04 12:28:34,312.0,2014-10-04 12:30:31,300.0,2014-10-04 12:31:30,305.0,2014-10-04 12:32:31,309.0,2014-10-04 12:34:31
2,838,2014-12-02 09:20:37,504.0,2014-12-02 09:20:38,68.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:38,838.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:40,838.0,2014-12-02 09:20:41,886.0,2014-12-02 09:20:42,27.0,2014-12-02 09:20:42,305.0,2014-12-02 09:20:42
3,190,2014-10-01 09:27:38,192.0,2014-10-01 09:27:38,8.0,2014-10-01 09:27:38,189.0,2014-10-01 09:27:38,191.0,2014-10-01 09:27:39,189.0,2014-10-01 09:27:39,190.0,2014-10-01 09:27:39,2375.0,2014-10-01 09:27:39,192.0,2014-10-01 09:27:39,8.0,2014-10-01 09:27:39
4,295,2014-10-02 14:34:17,295.0,2014-10-02 14:34:18,295.0,2014-10-02 14:34:19,295.0,2014-10-02 14:34:20,295.0,2014-10-02 14:34:21,295.0,2014-10-02 14:34:22,295.0,2014-10-02 14:34:23,295.0,2014-10-02 14:34:24,295.0,2014-10-02 14:34:25,295.0,2014-10-02 14:34:26
5,31,2014-05-19 17:50:21,3177.0,2014-05-19 17:50:21,3174.0,2014-05-19 17:50:21,32434.0,2014-05-19 17:50:22,27.0,2014-05-19 17:50:22,31.0,2014-05-19 17:50:22,5698.0,2014-05-19 17:50:25,5698.0,2014-05-19 17:50:26,5698.0,2014-05-19 17:50:27,4173.0,2014-05-19 17:50:27


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46473 entries, 1 to 46473
Data columns (total 20 columns):
site1     46473 non-null int64
time1     46473 non-null object
site2     46048 non-null float64
time2     46048 non-null object
site3     45643 non-null float64
time3     45643 non-null object
site4     45321 non-null float64
time4     45321 non-null object
site5     44985 non-null float64
time5     44985 non-null object
site6     44687 non-null float64
time6     44687 non-null object
site7     44401 non-null float64
time7     44401 non-null object
site8     44108 non-null float64
time8     44108 non-null object
site9     43828 non-null float64
time9     43828 non-null object
site10    43542 non-null float64
time10    43542 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 7.4+ MB


**В обучающей выборке – 400 пользователей.**

In [10]:
train_df['user_id'].nunique()

400

In [11]:
train_df['user_id'].value_counts().head()

149    4205
405    2592
984    2399
361    2297
884    2278
Name: user_id, dtype: int64

**Пока для прогноза ID пользователя будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [6]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [7]:
train_test_df_time = train_test_df[['time1', 'time2', 'time3', 
                                     'time4','time5', 
                                     'time6','time7', 'time8', 
                                     'time9', 'time10']].apply(pd.to_datetime)

In [13]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,23713,23720,23713,23713,23720,23713,23713,23713,23713,23713
2,8726,8725,665,8727,45,8725,45,5320,5320,5320
3,303,19,303,303,303,303,303,309,303,303
4,1359,925,1240,1360,1344,1359,1346,1345,1344,1345
5,11,85,52,85,11,52,11,85,10,85
6,83,1344,1240,1359,1345,85,1346,83,925,1359
7,13585,13585,13585,13585,13585,13585,13585,13585,13586,13585
8,29312,29311,29312,29312,0,0,0,0,0,0
9,21,62,18,1387,7,72,1386,1384,10,22


In [45]:
# calculate additional features
def calc_metrics(s, t):
    m1, m2, m3, m4, m5, m6 = 0, 0, 0, 0, 0, 0

    # 1. num of unique sites
    m1 = len(np.unique(s))

    t_min, t_max = min(t), max(t)
            
    # 2. start hour
    m2 = t_min.hour

    # 3. day of week
    m3 = t_min.weekday()
    
    # 4. month
    m4 = t_min.month
    
    # 5. day
    m5 = t_min.day
    
    # 6. day
    m6 = t_min.year

    return [m1, m2, m3, m4, m5, m6]

In [46]:
%%time
train_test_new_feat = []
for ix in tqdm(range(0, train_test_df_time.shape[0])):
    s = train_test_df_sites.iloc[ix,:]
    t = train_test_df_time.iloc[ix,:]
    train_test_new_feat.append(calc_metrics(s,t))

100%|████████████████████████████████████████████████████████████████████████████| 229266/229266 [01:54<00:00, 2000.86it/s]


Wall time: 1min 54s


In [47]:
with open(os.path.join(PATH_TO_DATA, 'new_features_400users.pkl'), 'wb') as new_features_400users_pkl:
    pickle.dump(train_test_new_feat, new_features_400users_pkl, protocol=2)

In [5]:
with open(os.path.join(PATH_TO_DATA, 'new_features_400users.pkl'), 'rb') as new_features_400users_pkl:
    train_test_new_feat = pickle.load(new_features_400users_pkl)

In [48]:
train_test_new_feat[:5]

[[2, 15, 0, 3, 24, 2014],
 [6, 14, 3, 4, 17, 2014],
 [3, 10, 4, 3, 21, 2014],
 [7, 9, 4, 12, 13, 2013],
 [4, 12, 1, 11, 26, 2013]]

**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [12]:
def convert_to_csr(M):
    indptr = [0]
    indices = []
    data = []

    # calculate site's number in each session
    for row_ix in tqdm(range(0, M.shape[0])):
        row = M[row_ix, :]
        s_ind, s_count = np.unique(row, return_counts=True)

        data.extend(s_count.tolist())
        indices.extend(s_ind.tolist())
        indptr.append(len(indices))

    return csr_matrix((data, indices, indptr), dtype=int)[:, 1:]

In [13]:
%%time
X_train_test_sparse = convert_to_csr(train_test_df_sites.values)

100%|███████████████████████████████████████████████████████████████████████████| 229266/229266 [00:03<00:00, 57735.08it/s]


Wall time: 4.31 s


In [14]:
X_train_sparse = X_train_test_sparse[:train_df.shape[0], :]

In [15]:
X_test_sparse = X_train_test_sparse[train_df.shape[0]:, :]

In [16]:
y = train_df.user_id.astype(int)

### Save data sets

In [17]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_train_400users.pkl'), 'wb') as pkl:
    pickle.dump(X_train_sparse, pkl, protocol=2)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_test_400users.pkl'), 'wb') as pkl:
    pickle.dump(X_test_sparse, pkl, protocol=2)

with open(os.path.join(PATH_TO_DATA, 'y_400users.pkl'), 'wb') as pkl:
    pickle.dump(y, pkl, protocol=2)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_400users.pkl'), 'wb') as pkl:
    pickle.dump(X_train_test_sparse, pkl, protocol=2)

### Load data sets

In [21]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_test_400users.pkl'), 'rb') as pkl:
    X_test_sparse = pickle.load(pkl)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_train_400users.pkl'), 'rb') as pkl:
    X_train_sparse = pickle.load(pkl)

with open(os.path.join(PATH_TO_DATA, 'y_400users.pkl'), 'rb') as pkl:
    y = pickle.load(pkl)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_400users.pkl'), 'rb') as pkl:
    X_train_test_sparse = pickle.load(pkl)

**1. Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [18]:
print("{0[0]} {0[1]} {1[0]} {1[1]}".format(X_train_sparse.shape, X_test_sparse.shape))

182793 36656 46473 36656


**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *train_target.pkl*).**

In [30]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3.**

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=17, stratify=y)

**Создайте объекты `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и с *hinge loss* (логистическая регрессия и линейный SVM соответственно) и параметром `random_state`=17. Остальные параметры оставьте по умолчанию, разве что `n_jobs`=-1 никогда не помешает. Обучите  модели на выборке `(X_train, y_train)`.**

In [23]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)

In [17]:
%%time
sgd_logit.fit(X_train, y_train)

Wall time: 33.9 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False)

In [18]:
sgd_svm = SGDClassifier(loss='hinge', random_state=17, n_jobs=-1)

In [19]:
%%time
sgd_svm.fit(X_train, y_train)

Wall time: 26.5 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False)

**Сделаем прогнозы с помощью обеих моделей на отложенной выборке *(X_valid, y_valid)*.**

In [20]:
logit_valid_pred = sgd_logit.predict(X_valid)
svm_valid_pred = sgd_svm.predict(X_valid)

**2. Напечатайте через пробел доли правильных ответов логистической регрессии и линейного SVM, обученных с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [21]:
print("%.3f" % accuracy_score(y_valid, logit_valid_pred), "%.3f" % accuracy_score(y_valid, svm_valid_pred))

0.302 0.295


**Сделайте прогноз для тестовой выборки с помощью *sgd_logit*.**

In [49]:
logit_test_pred = sgd_logit.predict(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Далее дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] _Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/identify-me-if-you-can4/leaderboard).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить, дополнительные баллы будут для тех, кто попадет в топ-10 и топ-30 по итогам этой недели.**

In [36]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [52]:
write_to_submission_file(logit_test_pred, '20170812-01.csv')

# Cross validation and paramers tuning for SGD, RF, GB, ExtraTrees

In [28]:
skf = StratifiedKFold(n_splits=3, random_state=17)

## SGD Logit

In [45]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1, alpha=1e-5, n_iter=15)

In [66]:
sgd_svm = SGDClassifier(loss='hinge', random_state=17, n_jobs=-1)

In [46]:
%%time
scores = cross_val_score(sgd_logit, X_train_sparse, y, scoring='accuracy', cv=skf, n_jobs=-1)

Wall time: 4min 9s


In [47]:
scores.mean()

0.33814784172019224

In [67]:
sgd_svm.get_params().keys()

['warm_start',
 'loss',
 'n_jobs',
 'eta0',
 'verbose',
 'shuffle',
 'fit_intercept',
 'epsilon',
 'average',
 'n_iter',
 'penalty',
 'power_t',
 'random_state',
 'l1_ratio',
 'alpha',
 'learning_rate',
 'class_weight']

In [41]:
parameters_grid = {
    'alpha' : [0.000001, 0.00001, 0.0001],
    #'eta0' : [0, 0.001, 0.05],
    'n_iter' : [5, 10, 15],
    'power_t' : [0.2, 0.5, 1, 2] 
}

In [42]:
grid_cv = GridSearchCV(SGDClassifier(loss='log', random_state=17, n_jobs=-1), parameters_grid, scoring = 'accuracy', cv = skf)

In [43]:
%%time
grid_cv.fit(X_train_sparse, y)

Wall time: 11h 48min 42s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=17, shuffle=False),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-06, 1e-05, 0.0001], 'power_t': [0.2, 0.5, 1, 2], 'n_iter': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [44]:
grid_cv.grid_scores_

[mean: 0.26078, std: 0.00324, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 5},
 mean: 0.26078, std: 0.00324, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 5},
 mean: 0.26078, std: 0.00324, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 5},
 mean: 0.26078, std: 0.00324, params: {'alpha': 1e-06, 'power_t': 2, 'n_iter': 5},
 mean: 0.28139, std: 0.00472, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 10},
 mean: 0.28139, std: 0.00472, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 10},
 mean: 0.28139, std: 0.00472, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 10},
 mean: 0.28139, std: 0.00472, params: {'alpha': 1e-06, 'power_t': 2, 'n_iter': 10},
 mean: 0.29318, std: 0.00293, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 15},
 mean: 0.29318, std: 0.00293, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 15},
 mean: 0.29318, std: 0.00293, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 15},
 mean: 0.29318, std: 0.00293, params: {'alpha': 1e-06, 'power_t': 2,

## SGD SVM

In [68]:
grid_cv = GridSearchCV(SGDClassifier(loss='hinge', random_state=17, n_jobs=-1), parameters_grid, scoring = 'accuracy', cv = skf)

In [69]:
parameters_grid = {
    'alpha' : [0.000001, 0.00001, 0.0001],
    'eta0' : [0, 0.001, 0.05],
    'n_iter' : [5, 10, 15]
}

In [70]:
%%time
grid_cv.fit(X_train_sparse, y)

Wall time: 9h 39min 26s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=17, shuffle=False),
       error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [1e-06, 1e-05, 0.0001], 'power_t': [0.2, 0.5, 1, 2], 'n_iter': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [71]:
grid_cv.grid_scores_

[mean: 0.25471, std: 0.00188, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 5},
 mean: 0.25471, std: 0.00188, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 5},
 mean: 0.25471, std: 0.00188, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 5},
 mean: 0.25471, std: 0.00188, params: {'alpha': 1e-06, 'power_t': 2, 'n_iter': 5},
 mean: 0.27360, std: 0.00257, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 10},
 mean: 0.27360, std: 0.00257, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 10},
 mean: 0.27360, std: 0.00257, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 10},
 mean: 0.27360, std: 0.00257, params: {'alpha': 1e-06, 'power_t': 2, 'n_iter': 10},
 mean: 0.28524, std: 0.00059, params: {'alpha': 1e-06, 'power_t': 0.2, 'n_iter': 15},
 mean: 0.28524, std: 0.00059, params: {'alpha': 1e-06, 'power_t': 0.5, 'n_iter': 15},
 mean: 0.28524, std: 0.00059, params: {'alpha': 1e-06, 'power_t': 1, 'n_iter': 15},
 mean: 0.28524, std: 0.00059, params: {'alpha': 1e-06, 'power_t': 2,

## Random Forest

In [72]:
forest = RandomForestClassifier(random_state=17, n_jobs=-1)

In [74]:
forest.get_params().keys()

['warm_start',
 'oob_score',
 'n_jobs',
 'verbose',
 'max_leaf_nodes',
 'bootstrap',
 'min_samples_leaf',
 'n_estimators',
 'min_samples_split',
 'min_weight_fraction_leaf',
 'criterion',
 'random_state',
 'min_impurity_split',
 'max_features',
 'max_depth',
 'class_weight']

In [75]:
parameters_grid = {
    'n_estimators' : [10, 20, 30],
    'bootstrap' : [True, False],
    'max_depth' : [10, 20, 30],
    'max_features' : ['sqrt', 'log2']
}

In [77]:
grid_cv = GridSearchCV(RandomForestClassifier(random_state=17, n_jobs=-1), parameters_grid, scoring = 'accuracy', cv = skf)

In [ ]:
%%time
grid_cv.fit(X_train_sparse, y)

Wall time: 7h 10min 10s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=17, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': ['sqrt', 'log2'], 'bootstrap': [True, False], 'max_depth': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [79]:
grid_cv.grid_scores_

[mean: 0.06949, std: 0.00143, params: {'max_features': 'sqrt', 'n_estimators': 10, 'bootstrap': True, 'max_depth': 10},
 mean: 0.08659, std: 0.00246, params: {'max_features': 'sqrt', 'n_estimators': 20, 'bootstrap': True, 'max_depth': 10},
 mean: 0.09666, std: 0.00134, params: {'max_features': 'sqrt', 'n_estimators': 30, 'bootstrap': True, 'max_depth': 10},
 mean: 0.03764, std: 0.00298, params: {'max_features': 'log2', 'n_estimators': 10, 'bootstrap': True, 'max_depth': 10},
 mean: 0.04791, std: 0.00326, params: {'max_features': 'log2', 'n_estimators': 20, 'bootstrap': True, 'max_depth': 10},
 mean: 0.05166, std: 0.00188, params: {'max_features': 'log2', 'n_estimators': 30, 'bootstrap': True, 'max_depth': 10},
 mean: 0.10004, std: 0.00155, params: {'max_features': 'sqrt', 'n_estimators': 10, 'bootstrap': True, 'max_depth': 20},
 mean: 0.12322, std: 0.00073, params: {'max_features': 'sqrt', 'n_estimators': 20, 'bootstrap': True, 'max_depth': 20},
 mean: 0.13139, std: 0.00018, params: {'

## Gradient Boosting

In [23]:
import xgboost
import lightgbm

In [28]:
xgb = xgboost.XGBClassifier(n_estimators=10, n_jobs=-1, silent=False)
X_trn, X_tst, y_trn, y_tst = train_test_split(X_train_sparse, y, test_size=0.3, random_state=9, stratify=y)

In [29]:
%%time
xgb.fit(X_trn, y_trn, verbose=True)

Wall time: 20min 5s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=-1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [30]:
accuracy_score(y_tst, xgb.predict(X_tst))

0.19222072285641342

In [58]:
lgb = lightgbm.LGBMClassifier(n_estimators=20, silent=False)

In [59]:
%%time
lgb.fit(X_trn.astype(float), y_trn.astype(float))

Wall time: 3min 47s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=20, nthread=-1,
        num_leaves=31, objective='multiclass', reg_alpha=0, reg_lambda=0,
        seed=0, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [60]:
accuracy_score(y_tst, lgb.predict(X_tst.astype(float)))

0.17847113315584084

# Пробуем добавить признаки

In [21]:
from collections import defaultdict, Counter

def prediction_bag(predictions, weights="uniform"):
    voting = []
    scores = defaultdict(list)
    
    weight_list = [1] * len(predictions)
    for i, prediction in enumerate(predictions):        
        if weights != "uniform":            
            weight_list[i] = weight_list[i] * weights[i]
        
        for ix, val in enumerate(prediction):
            scores[ix].extend(weight_list[i] * [val])
        
    for ix in sorted(scores):
        voting.append(Counter(scores[ix]).most_common(1)[0][0])
    
    return voting

def models_assessment(classifiers, X, y, test_size=0.3):
    cls_predictions = []
    X_trn, X_tst, y_trn, y_tst = X, X, y, y
    if test_size > 0:
        X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=test_size, random_state=17, stratify=y)
    
    for cls_estimator in classifiers:
        cls_name = type(cls_estimator)
        start_time = timeit.default_timer()
        cls_estimator.fit(X_trn, y_trn)
        y_cls = cls_estimator.predict(X_tst)
        cls_predictions.append(y_cls)
        elapsed = timeit.default_timer() - start_time
        print(cls_name, "%.3f" % accuracy_score(y_tst, y_cls), elapsed)
    
    return cls_predictions, y_tst

def make_predictions(classifiers, X):
    cls_predictions = []
    for cls_estimator in classifiers:
        cls_name = type(cls_estimator)
        start_time = timeit.default_timer()        
        y_cls = cls_estimator.predict(X)
        cls_predictions.append(y_cls)
        elapsed = timeit.default_timer() - start_time
        print(cls_name, elapsed)
    
    return cls_predictions


In [49]:
X_train_NF = np.array(train_test_new_feat)[:X_train_sparse.shape[0], :]
X_test_NF = np.array(train_test_new_feat)[X_train_sparse.shape[0]:, :]

# TRAIN DATASET
encoder = OneHotEncoder()
X_NFCat = encoder.fit_transform(X_train_NF[:,2:])
X_NFNonCat = X_train_NF[:,:2]

X_train_sparseNF = scipy.sparse.hstack([X_train_sparse, X_NFNonCat, X_NFCat])

In [50]:
skf = StratifiedKFold(n_splits=3, random_state=17)
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1, alpha=1e-5, n_iter=15)

In [53]:
%%time
scores = cross_val_score(sgd_logit, X_train_sparseNF, y, 
                         scoring='accuracy', cv=skf, n_jobs=-1)

Wall time: 3min 23s


In [54]:
scores.mean()

0.35661514631470731

In [61]:
%%time
sgd_logit.fit(X_train_sparseNF, y)

Wall time: 2min 31s


SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=15, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=17, shuffle=True, verbose=0,
       warm_start=False)

In [68]:
# prepare test dataset
# TEST DATASET
X_NFNonCat = X_test_NF[:,:2]
X_NFCat = encoder.transform(X_test_NF[:,2:])

X_test_sparseNF = scipy.sparse.hstack([X_test_sparse, X_NFNonCat, X_NFCat])

In [64]:
%%time
logit_test_predNF = sgd_logit.predict(X_test_sparseNF)

Wall time: 621 ms


In [65]:
write_to_submission_file(logit_test_predNF, '20170819-01.csv')

# Trying ensembles with basic classifiers from week 4

In [12]:
forest = RandomForestClassifier(n_estimators=25, random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1, , alpha=1e-5, n_iter=15)
sgd_svm = SGDClassifier(loss='hinge', random_state=17, n_jobs=-1, alpha=1e-4)

In [79]:
X_train150, X_test150, y_train150, y_test150 = train_test_split(X_150sparse, y_150, test_size=0.3, 
                                                                random_state=17, stratify=y_150)

In [80]:
classifiers=[('log', sgd_logit), ('svm', sgd_svm), ('mod_hub', sgd_hub), ('shinge', sgd_shin), ('perc', sgd_perc)]

In [90]:
%%time
cls_predictions = models_assessment(classifiers, X_150sparse, y_150)

log 0.426 6.03260430769
svm 0.420 5.062624
mod_hub 0.415 5.32296984615
shinge 0.396 5.27191876923
perc 0.390 5.11173415384
Wall time: 27 s


In [97]:
y_vote = prediction_bag(cls_predictions, weights=[3,1,1,1,1])

In [108]:
print("%.3f" % accuracy_score(y_test150, y_vote))

0.441


In [133]:
%%time

# now let's make another submission
classifiers=[('log', sgd_logit), ('svm', sgd_svm), ('mod_hub', sgd_hub), ('shinge', sgd_shin), ('perc', sgd_perc),
            ('forest', forest)]
cls_predictions = []
for cls_name, cls_estimator in classifiers:
    start_time = timeit.default_timer()
    cls_estimator.fit(X_train, y_train)
    y_cls = cls_estimator.predict(X_valid)
    cls_predictions.append(y_cls)
    elapsed = timeit.default_timer() - start_time
    print(cls_name, "%.3f" % accuracy_score(y_valid, y_cls), elapsed)

log 0.302 19.2451688205
svm 0.295 15.9992172308
mod_hub 0.289 16.9515798974
shinge 0.203 15.8908988718
perc 0.255 15.7406711795
forest 0.315 864.801350154
Wall time: 15min 48s


In [138]:
y_vote = prediction_bag(cls_predictions, weights=[2,1,1,1,1,3])
print("%.3f" % accuracy_score(y_valid, y_vote))

0.333


In [139]:
%%time

# make predictions
classifiers=[('log', sgd_logit), ('svm', sgd_svm), ('mod_hub', sgd_hub), ('shinge', sgd_shin), ('perc', sgd_perc),
            ('forest', forest)]
cls_predictions = []
for cls_name, cls_estimator in classifiers:
    start_time = timeit.default_timer()
    y_cls = cls_estimator.predict(X_test_sparse)
    cls_predictions.append(y_cls)
    elapsed = timeit.default_timer() - start_time
    print(cls_name, elapsed)

log 1.46372389744
svm 0.502006564104
mod_hub 0.510998153848
shinge 0.460102974357
perc 0.456911999998
forest 14.1221083077
Wall time: 17.5 s


In [113]:
y_vote = prediction_bag(cls_predictions, weights=[2,1,1,1,1,3])

In [144]:
write_to_submission_file(np.array(y_vote), '20170815-01.csv')

# 2017-08-23 submission

In [22]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1, alpha=1e-5, n_iter=15)
sgd_svm = SGDClassifier(loss='hinge', random_state=17, n_jobs=-1, alpha=1e-4, n_iter=15)
sgd_hub = SGDClassifier(loss='modified_huber', random_state=17, n_jobs=-1)
xgb = xgboost.XGBClassifier(n_estimators=15, n_jobs=-1, silent=False)
lgb = lightgbm.LGBMClassifier(n_estimators=50, silent=False, nthread=4)

In [23]:
classifiers=[sgd_logit, sgd_svm, sgd_hub, xgb, lgb]

In [155]:
%%time
cls_predictions, y_verify = models_assessment(classifiers, X_train_sparse.astype(float), y.astype(float), test_size=0)

<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.439 86.4972931304
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.409 72.3206147113
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.400 24.7632088097
<class 'xgboost.sklearn.XGBClassifier'> 0.243 2712.53635525
<class 'lightgbm.sklearn.LGBMClassifier'> 0.281 961.209285982
Wall time: 1h 4min 17s


In [156]:
y_vote = prediction_bag(cls_predictions, weights=[1,1,1,1,1])
print("%.3f" % accuracy_score(y_verify, y_vote))

0.452


In [157]:
%%time
cls_predictions2 = make_predictions(classifiers, X_test_sparse.astype(float))

<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.570333480748
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.495589336955
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.479659093617
<class 'xgboost.sklearn.XGBClassifier'> 28.7180703852
<class 'lightgbm.sklearn.LGBMClassifier'> 72.4175323754
Wall time: 1min 42s


In [158]:
y_vote = prediction_bag(cls_predictions2, weights=[1,1,1,1,1])

In [161]:
write_to_submission_file(np.array(y_vote, dtype=int), '20170822-01.csv')

In [24]:
def make_predictions_blend(clfs, X, y, X_submission, n_folds=3, n_classes=400):
    skf = list(StratifiedKFold(n_folds).split(X, y))
    dataset_blend_train = np.zeros((X.shape[0], len(clfs) * n_classes))
    dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs) * n_classes))

    for j, clf in enumerate(clfs):
        print("Classifier", j, type(clf))
        dataset_blend_test_j = np.zeros((X_submission.shape[0],  n_classes))
        
        for i, (train, test) in enumerate(skf):
            print("Fold", i)
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)
            dataset_blend_train[test, j*n_classes:(j+1)*n_classes] = y_submission
            dataset_blend_test_j = dataset_blend_test_j + (1./n_folds) * clf.predict_proba(X_submission)
        
        dataset_blend_test[:, j*n_classes:(j+1)*n_classes] = dataset_blend_test_j

    print()
    print("Blending...")
    clf = SGDClassifier(n_jobs=-1)
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict(dataset_blend_test)
    
    return y_submission

In [83]:
def make_predictions_blend2(clfs, X, y, X_submission, n_folds=3, n_classes=400):
    skf = list(StratifiedKFold(n_folds).split(X, y))
    dataset_blend_train = np.zeros((X.shape[0], len(clfs) * n_classes))
    dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs) * n_classes))

    for j, clf in enumerate(clfs):
        print("Classifier", j, type(clf), datetime.now())
        dataset_blend_test_j = np.zeros((X_submission.shape[0],  n_classes))
        
        for i, (train, test) in enumerate(skf):
            print("Fold", i, datetime.now())
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)
            dataset_blend_train[test, j*n_classes:(j+1)*n_classes] = y_submission
            dataset_blend_test_j = dataset_blend_test_j + (1./n_folds) * clf.predict_proba(X_submission)
        
        dataset_blend_test[:, j*n_classes:(j+1)*n_classes] = dataset_blend_test_j

    print()
    print("Blending...", datetime.now())
    clf = LogisticRegression(n_jobs=4)
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict(dataset_blend_test)
    
    return y_submission

In [92]:
%%time
y_blend = make_predictions_blend2([sgd_logit, sgd_hub, lgb, forest_g], 
                                 X_train_sparse.astype(float), np.array(y).astype(float), 
                                 X_test_sparse.astype(float), n_classes=400)

Classifier 0 <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 2017-08-23 10:23:11.013000
Fold 0 2017-08-23 10:23:11.026000
Fold 1 2017-08-23 10:24:09.373000
Fold 2 2017-08-23 10:25:07.244000
Classifier 1 <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 2017-08-23 10:26:04.172000
Fold 0 2017-08-23 10:26:04.202000
Fold 1 2017-08-23 10:26:49.232000
Fold 2 2017-08-23 10:27:34.587000
Classifier 2 <class 'lightgbm.sklearn.LGBMClassifier'> 2017-08-23 10:28:19.281000
Fold 0 2017-08-23 10:28:19.319000
Fold 1 2017-08-23 10:39:05.794000
Fold 2 2017-08-23 10:50:02.562000
Classifier 3 <class 'sklearn.ensemble.forest.RandomForestClassifier'> 2017-08-23 11:00:45.415000
Fold 0 2017-08-23 11:00:45.458000
Fold 1 2017-08-23 11:06:13.603000
Fold 2 2017-08-23 11:11:27.837000

Blending... 2017-08-23 11:17:08.378000
Wall time: 5h 10min 7s


In [93]:
write_to_submission_file(np.array(y_blend.astype(int)), '20170823-01.csv')

In [77]:
%%time
y_blend = make_predictions_blend2([sgd_logit, sgd_hub, lgb], X_150sparse.astype(float), np.array(y_150).astype(float), 
                                 X_150sparse.astype(float), n_classes=150)

Classifier 0 <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
Fold 0
Fold 1
Fold 2
Classifier 1 <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
Fold 0
Fold 1
Fold 2
Classifier 2 <class 'lightgbm.sklearn.LGBMClassifier'>
Fold 0
Fold 1
Fold 2

Blending...
Wall time: 45min 36s


In [78]:
print("%.3f" % accuracy_score(y_150.astype(float), y_blend))

0.510


# Trying ensembles with new features for 150 users

In [27]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'rb') as Sparse150users_pkl:
    X_150sparse = pickle.load(Sparse150users_pkl)

with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'rb') as y150users_pkl:
    y_150 = pickle.load(y150users_pkl)

In [89]:
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1, alpha=1e-5, n_iter=15)
sgd_svm = SGDClassifier(loss='hinge', random_state=17, n_jobs=-1, alpha=1e-4, n_iter=15)
sgd_hub = SGDClassifier(loss='modified_huber', random_state=17, n_jobs=-1, n_iter=15)
xgb = xgboost.XGBClassifier(n_estimators=15, n_jobs=-1, silent=False, max_depth=6)
lgb = lightgbm.LGBMClassifier(n_estimators=50, silent=False, nthread=4)
forest_g = RandomForestClassifier(n_estimators=20, n_jobs=-1, criterion='gini')
forest_en = RandomForestClassifier(n_estimators=15, n_jobs=-1, criterion='entropy')
extree_g = ExtraTreesClassifier(n_estimators=15, n_jobs=-1, criterion='gini')
extree_en = ExtraTreesClassifier(n_estimators=15, n_jobs=-1, criterion='entropy')

In [86]:
classifiers=[sgd_logit, sgd_hub, lgb, forest_g]

In [90]:
%%time
cls_predictions, y_verify = models_assessment(classifiers, X_150sparse.astype(float), y_150.astype(float))

<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.444 16.2292773503
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> 0.437 13.7105602375
<class 'lightgbm.sklearn.LGBMClassifier'> 0.351 356.914311865
<class 'sklearn.ensemble.forest.RandomForestClassifier'> 0.393 129.855878682
Wall time: 8min 36s


In [91]:
y_vote = prediction_bag(cls_predictions, weights=[1,1,1,1])
print("%.3f" % accuracy_score(y_verify, y_vote))

0.461


In [71]:
print(np.corrcoef(cls_predictions))

[[ 1.          0.66901575  0.55994654  0.49936139  0.57015764  0.50373599
   0.58402801  0.53919013]
 [ 0.66901575  1.          0.52737125  0.47089862  0.54017446  0.47493743
   0.55401939  0.51538461]
 [ 0.55994654  0.52737125  1.          0.65907508  0.59093243  0.54859315
   0.58538723  0.56251848]
 [ 0.49936139  0.47089862  0.65907508  1.          0.54374809  0.51405253
   0.53672738  0.52286698]
 [ 0.57015764  0.54017446  0.59093243  0.54374809  1.          0.63093381
   0.69889343  0.66061194]
 [ 0.50373599  0.47493743  0.54859315  0.51405253  0.63093381  1.
   0.62535538  0.63835554]
 [ 0.58402801  0.55401939  0.58538723  0.53672738  0.69889343  0.62535538
   1.          0.68671016]
 [ 0.53919013  0.51538461  0.56251848  0.52286698  0.66061194  0.63835554
   0.68671016  1.        ]]


In [30]:
%%time
cls_predictions, y_verify = models_assessment([('adaboost', AdaBoostClassifier(
    base_estimator=SGDClassifier(loss='log', random_state=17, n_jobs=-1), random_state=17))], X_150sparseNFScaled, y_150)

adaboost 0.453 450.717513026
Wall time: 7min 31s


# Выводы

Пользователь в Kaggle - [YDF & MIPT] _VladNF Public score: 0.20395 (3rd result at 3-45 pm Aug-23)

1. Среди базовых классификаторов наилучшее качество на тестовой выборке показал SGDClassifier, после оптимизации параметров - позволяет преодолеть бейзлайн.
2. Несмотря на то, что feature engineering давал существенное улучшение на train, на тесте результаты были существенно хуже. В итоге лучший результат был получен без генерации новых признаков
3. Смешивание алгоритмов - алгоритм голосования, один из самых простых - позволил улучшить качество. Наилучший результат был получен с помощью приема blending из статьи по ссылке - код реализации make_predictions_blend2

## Критерии оценки работы:
-  Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
-  Правильные ли получились доли правильных ответов логистической регрессии и линейного SVM в п . 2? (max. 4 балла)
-  Каков результат лучшей из посылок на Kaggle, побит ли бейзлайн SGDCLassifier?  (max. 2 балла)
-  Входит ли  посылка в топ-30 лучших на данный момент на публичном лидерборде соревнования? (max. 2 балла)
-  Входит ли  посылка в топ-10 лучших на данный момент на публичном лидерборде соревнования? (max. 4 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных 400 пользователями веб-страницах (400 csv-файлов в *train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.